In [1]:
## Import

%matplotlib inline
import numpy as np
import pandas as pd
import os
import urllib.request
import datetime
import gzip
import shutil
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from math import cos, asin, sqrt
import datetime
import sklearn
import sys

listing_col  = ['id', ## om het nog te traceren
               'host_id',## om het nog te traceren
               'listing_url',
               'name',
               'price',
#                'summary', ## Heel vaak niet gevuld, heeft te veel invloed op het uiteindelijke resultaat
               'host_name',
#                'host_about',
               'description',
               'neighbourhood_cleansed', # om het nog te aggrereen
#                'property_type',
               'room_type',
               'beds',
               'bedrooms',
               'bathrooms',
               'accommodates',
               'latitude',  # nog even erin laten
               'longitude'  # nog even erin laten
              ]


In [2]:
##Handige cell om inzicht in groote van objecten te weergeven

sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules], key=lambda x: x[1], reverse=True)

[('TfidfVectorizer', 1056),
 ('Out', 240),
 ('listing_col', 184),
 ('cosine_similarity', 136),
 ('linear_kernel', 136),
 ('In', 96),
 ('np', 80),
 ('pd', 80),
 ('preprocessing', 80),
 ('sparse', 80),
 ('asin', 72),
 ('cos', 72),
 ('sqrt', 72),
 ('get_ipython', 64),
 ('exit', 56),
 ('quit', 56)]

## Loading all the data of 2018


url = 'http://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/2018-01-10/data/listings.csv.gz'
filename = 'listings_2018-01-10.csv.gz'
urllib.request.urlretrieve(url, filename)
unfilled = gzip.open(filename)
df_listings = pd.read_csv(unfilled, usecols=listing_col)
df_listings['publicatie'] = '2018-01-10'
item = "listings" 
            

our_dates= []
start = datetime.datetime.strptime("11-01-2018", "%d-%m-%Y")  # we can change this
end = datetime.datetime.strptime("31-12-2018", "%d-%m-%Y")    # we can change this
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]

for date in date_generated:
    our_dates.append(date.strftime("%Y-%m-%d"))

for date in our_dates:
    url = "http://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/" +date+ "/data/"+item+".csv.gz"
    filename = item +"_"+ date +".csv.gz" 
    file_python = item +"_"+ date
    try:
        urllib.request.urlretrieve(url, filename)
        print("found : " + date)
        unfilled = gzip.open(filename)
#         df_reviews[file_python] = pd.read_csv(unfilled)
        temp = pd.read_csv(unfilled, usecols=listing_col)
        temp['publicatie'] = date
        
        df_listings.append(temp)
        
        
    except urllib.error.URLError as e:
        print(e.reason) 
        


## Loading only one run

In [2]:
url = 'http://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/2019-01-13/data/listings.csv.gz'
filename = 'listings_2019-01-13.csv.gz'
urllib.request.urlretrieve(url, filename)
unfilled = gzip.open(filename)
df_listings = pd.read_csv(unfilled, usecols=listing_col)
# df_listings['publicatie'] = '2019-01-13'

del unfilled

In [5]:


(594 *2) /len(df_listings)

0.05966850828729282

In [7]:
len(df_listings)

19910

## Set verdeelsleutel

In [4]:
verdeelsleutel_description        =0.4
verdeelsleutel_name               =0.1
verdeelsleutel_price              =0.05
verdeelsleutel_location           =0.4
verdeelsleutel_listing_attributes =0.05
# 40%, name 10%, price 5%, location 40%, amenities 5

## Cleansing

In [5]:
print_helper = list(range(0, len(df_listings), 1000))
df_listings['bedrooms'].fillna((df_listings['bedrooms'].mean()), inplace=True)
df_listings['beds'].fillna((df_listings['beds'].mean()), inplace=True)
df_listings['bathrooms'].fillna((df_listings['bathrooms'].mean()), inplace=True)
df_listings['price'] = pd.to_numeric(df_listings['price'].str.replace(',','').str.replace('$',''))
df_listings.description = df_listings.description.fillna('')
df_listings.host_name = df_listings.host_name.fillna('')
df_listings.name = df_listings.name.fillna('')

## Column Description and Name

In [6]:
def tfidf_cosine_function (columname, listing_id_column, verdeel_sleutel= 1):
    """ This function turns a non numerical column into a cosine simalartiy matrix using tfidf """
    
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(columname)

    cosine_simalarity_array = linear_kernel(tfidf_matrix,tfidf_matrix)
    np.fill_diagonal(cosine_simalarity_array, 0)
    output = pd.DataFrame(cosine_simalarity_array)
    output.rename(index=listing_id_column , inplace = True)
    output.rename(columns= listing_id_column, inplace = True)
    del cosine_simalarity_array      
    return output * verdeel_sleutel

          

In [7]:
description =tfidf_cosine_function(df_listings.description, df_listings.id,verdeelsleutel_description )
name        =tfidf_cosine_function(df_listings.name, df_listings.id , verdeelsleutel_name)

## Column Listing attributes

In [8]:
# Memommery issues door amnestiesen propertytype
listing_attributes = df_listings[['accommodates','bathrooms','bedrooms','beds', 'room_type']]
listing_attributes =pd.get_dummies(listing_attributes, columns=['room_type'])
cosine_simalarity_array = cosine_similarity(listing_attributes,listing_attributes)
np.fill_diagonal(cosine_simalarity_array, 0)
listing_attributes = pd.DataFrame(cosine_simalarity_array * verdeelsleutel_listing_attributes)
listing_attributes.rename(index= df_listings.id , inplace = True)
listing_attributes.rename(columns= df_listings.id, inplace = True)
del cosine_simalarity_array    

## Column Location

In [9]:
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295     #Pi/180
    a = 0.5 - cos((lat2 - lat1) * p)/2 + cos(lat1 * p) * cos(lat2 * p) * (1 - cos((lon2 - lon1) * p)) / 2
    return 12742 * asin(sqrt(a)) #2*R*asin...

# distance(52.36755,4.9414,53.390225,4.873924) ## voorbeeldje

tuple_set = list(zip(df_listings.latitude, df_listings.longitude)) ## list met alle tuples

matrix = np.zeros((len(tuple_set), len(tuple_set))) ## iniatilize the array

for i in range(len(tuple_set)):  
    for j in range(len(tuple_set)):
        matrix[i][j] = abs(distance(*tuple_set[i], *tuple_set[j]))
    if i in print_helper:
        print("Bezig met row ", i, "van de ", len(tuple_set), "current time: ",datetime.datetime.now())

        
loc = sklearn.preprocessing.minmax_scale(matrix, feature_range=(0, 1), axis=1, copy=True)
location = [1] - loc
np.fill_diagonal(location, 0)
location = pd.DataFrame(location * verdeelsleutel_location)       
location.rename(index= df_listings.id , inplace = True)
location.rename(columns= df_listings.id, inplace = True)

del distance
del matrix
del loc
del tuple_set


Bezig met row  0 van de  19910 current time:  2019-01-21 14:06:46.624158
Bezig met row  1000 van de  19910 current time:  2019-01-21 14:07:17.376906
Bezig met row  2000 van de  19910 current time:  2019-01-21 14:07:46.898949
Bezig met row  3000 van de  19910 current time:  2019-01-21 14:08:16.895719
Bezig met row  4000 van de  19910 current time:  2019-01-21 14:08:46.242231
Bezig met row  5000 van de  19910 current time:  2019-01-21 14:09:15.391270
Bezig met row  6000 van de  19910 current time:  2019-01-21 14:09:45.028005
Bezig met row  7000 van de  19910 current time:  2019-01-21 14:10:14.465271
Bezig met row  8000 van de  19910 current time:  2019-01-21 14:10:44.001275
Bezig met row  9000 van de  19910 current time:  2019-01-21 14:11:13.579168
Bezig met row  10000 van de  19910 current time:  2019-01-21 14:11:43.152112
Bezig met row  11000 van de  19910 current time:  2019-01-21 14:12:12.750910
Bezig met row  12000 van de  19910 current time:  2019-01-21 14:12:42.092432
Bezig met ro

## Column Price

In [10]:
extractedData = df_listings.loc[:,['price']].values


# Uitgezet wegens performance redenen
# for i in range(1,len(extractedData)):
#     newrow = abs(extractedData.T - extractedData[i])
#     A = np.vstack([A, newrow])
#     if i in print_helper:
#         print("Bezig met row ", i, "van de ", len(extractedData), "current time: ",datetime.datetime.now())

matrix2 = np.zeros((len(extractedData), len(extractedData))) ## iniatilize the array
        
for i in range(len(extractedData)):  
    for j in range(len(extractedData)):
        matrix2[i][j] = abs(extractedData[i] - extractedData[j])
    if i in print_helper:
        print("Bezig met row ", i, "van de ", len(extractedData), "current time: ",datetime.datetime.now())

price = sklearn.preprocessing.minmax_scale(matrix2, feature_range=(0, 1), axis=1, copy=True)
price = [1] - price
np.fill_diagonal(price, 0)
price = pd.DataFrame(price * verdeelsleutel_price)  
price.rename(index= df_listings.id , inplace = True)
price.rename(columns= df_listings.id, inplace = True)

del matrix2
del extractedData

Bezig met row  0 van de  19910 current time:  2019-01-21 14:17:11.236609
Bezig met row  1000 van de  19910 current time:  2019-01-21 14:17:42.238735
Bezig met row  2000 van de  19910 current time:  2019-01-21 14:18:12.979476
Bezig met row  3000 van de  19910 current time:  2019-01-21 14:18:43.724243
Bezig met row  4000 van de  19910 current time:  2019-01-21 14:19:14.688427
Bezig met row  5000 van de  19910 current time:  2019-01-21 14:19:45.248696
Bezig met row  6000 van de  19910 current time:  2019-01-21 14:20:16.437315
Bezig met row  7000 van de  19910 current time:  2019-01-21 14:20:47.199999
Bezig met row  8000 van de  19910 current time:  2019-01-21 14:21:17.806146
Bezig met row  9000 van de  19910 current time:  2019-01-21 14:21:48.643666
Bezig met row  10000 van de  19910 current time:  2019-01-21 14:22:19.648741
Bezig met row  11000 van de  19910 current time:  2019-01-21 14:22:50.170107
Bezig met row  12000 van de  19910 current time:  2019-01-21 14:23:20.763285
Bezig met ro

## Add all the frames together in one frame

In [11]:
# Helaas loop ik tegen memomery issues aan, daarom verwijder ik de helft van elke frame, deze zou daarna opnieuw gedraaid moeten worden

eerste_helft = 0
tweede_helft = 0
derde_helft  = 1

if eerste_helft == 1:
    description        = description.iloc[0:8000,:]
    price              = price.iloc[0:8000,:]
    listing_attributes = listing_attributes.iloc[0:8000,:]
    location           = location.iloc[0:8000,:]
    name               = name.iloc[0:8000,:]

if tweede_helft == 1:
    description        = description.iloc[8000:15000,]
    price              = price.iloc[8000:15000,]
    listing_attributes = listing_attributes.iloc[8000:15000,]
    location           = location.iloc[8000:15000,]
    name               = name.iloc[8000:15000,]

if derde_helft == 1:
    description        = description.iloc[15000:,:]
    price              = price.iloc[15000:,:]
    listing_attributes = listing_attributes.iloc[15000:,:]
    location           = location.iloc[15000:,:]
    name               = name.iloc[15000:,:]
        

In [12]:
# result = description + name + price + location + listing_attributes
result = description.add(name)
result = result.add(location)
result = result.add(price)
result = result.add(listing_attributes)

## Selecting the highest score per id

In [13]:
best_matching_id = result.idxmax()
match_value = result.max(axis=1)
# del result
match = pd.DataFrame({'best_matching_id':best_matching_id, 'match_value':match_value})
match =match.sort_values(by=['match_value'], ascending=False)
match_df = match.sort_values(by=['match_value'], ascending=False).reset_index()
match_df.rename(columns = {'index':'id'}, inplace = True)

## Enkel de matches overhouden met match_value > 70
match_df = match_df[match_df['match_value'] > 0.7]


## Only to show the urls

result = pd.merge(match_df,
                 df_listings[['id', 'listing_url', 'host_id']],
                 on='id')
result.head()

result = pd.merge(result,
                 df_listings[['id', 'listing_url', 'host_id']],
                 left_on='best_matching_id', right_on = 'id')


result.head()

result_filter = result[result.host_id_x != result.host_id_y]

def make_clickable(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}">{}</a>'.format(val, val)


result_filter = result_filter.sort_values(by=['match_value'], ascending=False)
result_filter.style.format({'listing_url_x': make_clickable,'listing_url_y': make_clickable })


In [14]:
# Find corresponding feature contributions
list_holder_attr  = []
list_holder_price = []
list_holder_name  = []
list_holder_desc  = []
list_holder_loc   = []

for index, row in match_df.iterrows():
    idnr = row['id']
    best_match = row['best_matching_id']
    list_holder_price.append(round(price.loc[idnr,best_match]/verdeelsleutel_price,3))
    list_holder_loc.append(round(location.loc[idnr,best_match]/verdeelsleutel_location,3))
    list_holder_name.append(round(name.loc[idnr,best_match]/verdeelsleutel_name,3))
    list_holder_attr.append(round(listing_attributes.loc[idnr,best_match]/verdeelsleutel_listing_attributes,3))
    list_holder_desc.append(round(description.loc[idnr,best_match] / verdeelsleutel_description,3))


    
    

In [15]:
match_df['feature_listing_attributes'] = list_holder_attr
match_df['feature_price'] = list_holder_price 
match_df['feature_name'] = list_holder_name  
match_df['feature_description'] = list_holder_desc  
match_df['feature_location'] = list_holder_loc   
match_df['duplicate_ID'] = range(1, len(match_df) + 1)
match_df['type']  = 'listings'
match_df['status']  = 'Undifined'
match_df['timestamp'] = "13-01-2019"
match_df.rename(columns = {'id':'case_A_id','best_matching_id':'case_B_id','match_value':'match_score'}, inplace = True)
output =match_df[["duplicate_ID","type","case_A_id","case_B_id","match_score", "status", "timestamp", "feature_listing_attributes",'feature_price','feature_name','feature_location','feature_description']]


In [16]:
y_cond = (output.case_A_id.shift(1) == output.case_B_id)
output['Result'] = np.where(y_cond, 'Y', 'N')
output = output[output['Result']== 'Y' ]
output =output.iloc[:,:-1]  ## Remove the result column , which indicates if their is a dubblicate match



C:\Users\Jeffr\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [17]:
output

,duplicate_ID,type,case_A_id,case_B_id,match_score,status,timestamp,feature_listing_attributes,feature_price,feature_name,feature_location,feature_description
1,2,listings,22556265,22556393,0.999184,Undifined,13-01-2019,1.000,1.000,1.000,0.998,1.000
3,4,listings,25177209,30976179,0.999034,Undifined,13-01-2019,1.000,1.000,1.000,0.998,1.000
5,6,listings,23855455,24417774,0.997385,Undifined,13-01-2019,0.988,0.999,1.000,0.995,1.000
16,17,listings,23500074,31150995,0.995665,Undifined,13-01-2019,1.000,1.000,1.000,0.989,1.000
18,19,listings,31146480,28688750,0.995441,Undifined,13-01-2019,1.000,1.000,1.000,0.989,1.000
20,21,listings,23061996,23880524,0.995216,Undifined,13-01-2019,1.000,1.000,1.000,0.988,1.000
23,24,listings,23956368,26943704,0.995100,Undifined,13-01-2019,1.000,0.996,1.000,0.988,1.000
25,26,listings,23777372,23626417,0.994863,Undifined,13-01-2019,1.000,1.000,1.000,0.987,1.000
31,32,listings,23874962,24036014,0.993906,Undifined,13-01-2019,1.000,0.994,1.000,0.986,1.000
33,34,listings,29836165,30514273,0.993879,Undifined,13-01-2019,1.000,1.000,0.981,0.989,1.000


In [18]:
df_listings[df_listings['id']== 22556265]


,id,listing_url,name,description,host_id,host_name,neighbourhood_cleansed,latitude,longitude,room_type,accommodates,bathrooms,bedrooms,beds,price
15096,22556265,https://www.airbnb.com/rooms/22556265,Central & cosy ground floor stay Vondelpark 1,This cosy and central stay is located at the V...,165633227,Anisha,De Baarsjes - Oud-West,52.359363,4.858259,Private room,1,1.0,1.0,1.0,68.0


In [19]:

df_listings[df_listings['id']== 22556393]

,id,listing_url,name,description,host_id,host_name,neighbourhood_cleansed,latitude,longitude,room_type,accommodates,bathrooms,bedrooms,beds,price
15097,22556393,https://www.airbnb.com/rooms/22556393,Central & cosy ground floor stay Vondelpark 2,This cosy and central stay is located at the V...,165633227,Anisha,De Baarsjes - Oud-West,52.359348,4.857868,Private room,1,1.0,1.0,1.0,68.0


## JSON File

In [20]:
output.to_json('duplicates_nieuwe_verdeelsleutel3.js' ,orient='records')